In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
%pip install lyricsgenius
import lyricsgenius as lg

Note: you may need to restart the kernel to use updated packages.


In [7]:
songs = pd.read_csv("songs.csv", sep=";")
songs.head()

,title,artist,top genre,year,bpm,energy,danceability,dB,liveness,valence,duration,acousticness,speechiness,popularity
0,Flowers,Miley Cyrus,pop,2023,118,68,71,-4,3,65,200,6,7,98
1,Cupid - Twin Ver.,FIFTY FIFTY,k-pop girl group,2023,120,59,78,-8,35,73,174,44,3,97
2,BESO,ROSALÍA,pop,2023,95,64,77,-7,17,53,195,74,14,96
3,Boy's a liar Pt. 2,PinkPantheress,bronx drill,2023,133,81,70,-8,25,86,131,25,5,96
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,rap,2022,98,62,72,-6,8,17,222,42,5,96


In [8]:
def search_lyrics(song_title, artist_name):
    access_token = '8yvpmDv96aodI5vg660Afcby4XPdrhPrx4JCAM3souNcRYG9C2nF5TWg1'
    genius = lg.Genius(access_token)
    song = genius.search_song(song_title, artist_name)
    return song.lyrics if song is not None else None

In [9]:
# all_lyrics = [search_lyrics(songs.title[i], songs.artist[i]) for i in songs.iterrows()]
# songs.insert(loc=2, column="lyrics", value=pd.Series(data=all_lyrics, index=songs.index, dtype="object"))
# songs.to_csv("songs_with_lyrics.csv")

In [10]:
songs.columns = songs.columns.str.replace('speechiness ', 'speechiness').str.replace('danceability ', 'danceability')

In [15]:
corr = songs.corr(numeric_only=True)
print(abs(corr["popularity"]).sort_values(ascending=False))

popularity         1.000000
songs of artist    0.241812
year               0.205909
energy             0.120484
acousticness       0.100588
duration           0.095590
valence            0.086720
speechiness        0.072670
top genre          0.068774
bpm                0.019961
danceability       0.019491
liveness           0.019009
Name: popularity, dtype: float64


Come si può vedere dagli indici di correlazione delle features rispetto alla label *popularity*, i decibel *dB* non influiscono in maniera significativa sulla popolarità, quindi la colonna può essere eliminata (in realtà forse conviene comunque tenerla perché le varie regolarizzazioni sistemano già questa cosa).

In [12]:
songs.drop(["dB"], axis=1, inplace=True)

In [13]:
songs_of_artist = songs.groupby("artist").count().aggregate("max", axis=1)
songs["songs of artist"] = songs["artist"].map(songs_of_artist)

In [16]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
songs["top genre"] = label_encoder.fit_transform(songs["top genre"])

In [18]:
print(songs["year"].var(ddof=0))
print(songs["bpm"].var(ddof=0))
print(songs["energy"].var(ddof=0))
print(songs["danceability"].var(ddof=0))
print(songs["liveness"].var(ddof=0))
print(songs["valence"].var(ddof=0))
print(songs["duration"].var(ddof=0))
print(songs["acousticness"].var(ddof=0))
print(songs["speechiness"].var(ddof=0))
print(songs["popularity"].var(ddof=0))

45.521325896918626
743.5087472805665
261.1850164154899
188.13338009660305
185.67957948217588
504.1540666904
1867.7644600908015
411.0145899643562
90.60791828733919
130.35579007511132


In [19]:
# var() > 200

bpm = songs["bpm"].values.reshape(-1, 1)
energy = songs["energy"].values.reshape(-1, 1)
valence = songs["valence"].values.reshape(-1, 1)
duration = songs["duration"].values.reshape(-1, 1)
acousticness = songs["acousticness"].values.reshape(-1, 1)

############################################################################################################

# var() < 200

year = songs["year"].values.reshape(-1, 1)
danceability = songs["danceability"].values.reshape(-1, 1)
liveness = songs["liveness"].values.reshape(-1, 1)
speechiness = songs["speechiness"].values.reshape(-1, 1)
popularity = songs["popularity"].values.reshape(-1, 1)

In [20]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
minMaxScaler = MinMaxScaler()

In [21]:
# var() > 200
songs["bpm"] = standardScaler.fit_transform(bpm)
songs["energy"] = standardScaler.fit_transform(energy)
songs["valence"] = standardScaler.fit_transform(valence)
songs["duration"] = standardScaler.fit_transform(duration)
songs["acousticness"] = standardScaler.fit_transform(acousticness)

############################################################################################################

# var() <= 200
songs["year"] = minMaxScaler.fit_transform(year)
songs["danceability"] = minMaxScaler.fit_transform(danceability)
songs["liveness"] = minMaxScaler.fit_transform(liveness)
songs["speechiness"] = minMaxScaler.fit_transform(speechiness)
songs["popularity"] = minMaxScaler.fit_transform(popularity)

In [22]:
songs

,title,artist,top genre,year,bpm,energy,danceability,liveness,valence,duration,acousticness,speechiness,popularity,songs of artist
0,Flowers,Miley Cyrus,221,1.000000,-0.065413,-0.110911,0.6625,0.012195,0.479484,-0.600374,-0.474166,0.089286,1.000000,10
1,Cupid - Twin Ver.,FIFTY FIFTY,181,1.000000,0.007934,-0.667799,0.7500,0.402439,0.835778,-1.201980,1.400203,0.017857,0.989247,1
2,BESO,ROSALÍA,221,1.000000,-0.908913,-0.358417,0.7375,0.182927,-0.054957,-0.716068,2.879967,0.214286,0.978495,2
3,Boy's a liar Pt. 2,PinkPantheress,56,1.000000,0.484695,0.693484,0.6500,0.280488,1.414756,-2.196944,0.463018,0.053571,0.978495,1
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,230,0.973684,-0.798891,-0.482170,0.6750,0.073171,-1.658279,-0.091323,1.301552,0.053571,0.978495,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,Southbound,Beach Blvd,236,1.000000,0.741412,1.126620,0.5250,0.097561,-0.233104,0.047510,0.167066,0.035714,0.172043,1
2381,Dance with Somebody - Radio Version,Mando Diao,93,0.631579,1.108151,1.250373,0.4625,0.414634,-0.144030,0.348313,-0.770119,0.125000,0.118280,1
2382,Flow,Desire Machines,163,0.973684,-0.725543,1.064743,0.5250,0.085366,0.880315,0.672254,-0.770119,0.053571,0.107527,1
2383,Scared of the Dark,Everything Brighter,221,1.000000,0.007934,0.631607,0.5875,0.060976,0.301337,-1.086287,-0.770119,0.071429,0.064516,1
